In [2]:
import requests
import json

In [13]:
import tkinter
from tkinter import messagebox
import datetime
import smtplib
from tkinter import simpledialog

pwd = "llsx rwhp nfuf dpbe"

In [ ]:
window = tkinter.Tk()
window.minsize(width = 250, height = 250)
window.title("US Cities Weather")
window.config(bg = "#ADD8E6", padx = 10, pady = 10)

city_label = tkinter.Label()
city_label.config(text = "City:", bg = "white", font = ("Georgia", 12, "normal"), highlightthickness = 1, highlightbackground = "black")
city_label.grid(column = 1, row = 1)

city_text = tkinter.Text()
city_text.focus()
city_text.config(width = 20, height = 1, pady = 5)
city_text.grid(row = 1, column = 2, padx = 5)

weather_report = None

def get_weather():
    global weather_report
    city = city_text.get("1.0", tkinter.END)
    c = city.strip()
    parameters = {"q":f"{c.title()}", "appid": "da21cb59f268b9c3b67c070c0c219996"}
    resp = requests.get(url = "https://api.openweathermap.org/data/2.5/weather", params = parameters )
    try:
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        messagebox.showinfo(title = "Alert!", message = "Enter a city in the US")
        city_text.delete("1.0", tkinter.END)
        
    status_code = resp.status_code
    f = resp.json()
    if status_code == 200:
        latitude = f["coord"]["lat"]
        longitude = f["coord"]["lon"]
        weather = f["weather"][0]["description"]
        country = f["sys"]["country"]
        sunrise = f["sys"]["sunrise"]
        sunset = f["sys"]["sunset"]
        timezone = f["timezone"]
        temp = f["main"]["temp"]
        temp_min = f["main"]["temp_min"]
        temp_max = f["main"]["temp_max"]
        humidity = f["main"]["humidity"]
        idno = f["weather"][0]["id"]
        sunrise_time = datetime.datetime.fromtimestamp(sunrise)
        sunset_time = datetime.datetime.fromtimestamp(sunset)

        weather_report = f"City: {c.title()}\nCountry: {country}\nLatitude: {latitude}\nLongitude: {longitude}\nTemperature: {temp}\nMinimum: {temp_min}\nMaximum: {temp_max}\nHumidity: {humidity}\nWeather: {weather}\nTimezone: {timezone//3600} Hours\nSunrise Time: {sunrise_time}\nSunset Time: {sunset_time}"
        
        report_label = tkinter.Label()
        report_label.config(text = weather_report, bg = "white", font = ("Georgia", 8, "normal"), highlightthickness = 1, highlightbackground = "black")
        report_label.grid(column = 1, row = 3, columnspan = 2)
        
        if idno < 800:
            text_label = tkinter.Label()
            text_label.config(text = "Bring an umbrella!!", bg = "#ADD8E6", font = ("Georgia", 9 , "bold"), highlightthickness = 1, highlightbackground = "black")
            text_label.grid(column = 1, row = 4, columnspan = 2, pady = 10)
        
weather_button = tkinter.Button(text = "Get Report", command = get_weather)
weather_button.config(bg = "white", width = 10, font = ("Georgia" , 9, "normal"))
weather_button.grid(column = 1, row = 2, columnspan = 2, pady = 10)

def send_mail():
    global weather_report
    email = simpledialog.askstring("Input", "Enter your Email ID")
    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as con:
            con.starttls()
            con.login(user = "silvervoid3.14@gmail.com", password = pwd)
            con.sendmail(from_addr = "silvervoid3.14@gmail.com", to_addrs = email, msg = f"Subject:Weather Report\n\n{weather_report}")

        email_text_label = tkinter.Label()
        email_text_label.config(text = "Check your inbox!", bg = "#ADD8E6", font = ("Georgia" , 9, "normal"))
        email_text_label.grid(column = 1, row = 6, columnspan = 2)
        
    except smtplib.SMTPRecipientsRefused as e:
        messagebox.showinfo(title = "Alert!", message = "Enter a valid email!")

email_button = tkinter.Button(text = "Send Report", command = send_mail)
email_button.config(bg = "white",  width = 10, font = ("Georgia" , 9, "normal"))
email_button.grid(column = 1, row = 5, columnspan = 2, pady = 5)

window.mainloop()